In [2]:
import pandas as pd
import plotly.express as px

from glob import glob
from mainnet_launch.constants import *
from mainnet_launch.database.views import get_token_details_dict


def load_swap_matrix_with_prices_data():
    swap_matrix_data2 = WORKING_DATA_DIR / "swap_matrix_prices2"
    paths = [a for a in swap_matrix_data2.glob("*.csv")]

    dfs = [pd.read_csv(p) for p in paths]
    df = pd.concat(dfs).reset_index(drop=True)

    df = df[(df["3rd_party_response_success"] == True) & (df["prices_success"] == True)].copy()
    df["datetime_received"] = pd.to_datetime(df["datetime_received"], utc=True)

    df = add_actual_prices(df)

    interesting_columns = [
        "datetime_received",
        "aggregatorName",
        "autopool_name",
        "buy_symbol",
        "sell_symbol",
        "buy_amount_norm",
        "sell_amount_norm",
        "buy_amount_price",
        "buy_token_price",
        "sell_token_price",
        "label",
        "safe_value_slippage_bps",
        "long_label",
        "30_min_chunk",
        "60_min_chunk",
        "2_hour_chunk",
    ]
    return df[interesting_columns].reset_index(drop=True)


def add_actual_prices(df: pd.DataFrame) -> pd.DataFrame:
    token_to_decimals, token_to_symbol = get_token_details_dict()

    df["buy_amount_norm"] = df.apply(
        lambda row: int(row["buyAmount"]) / 10 ** token_to_decimals[row["buyToken"]], axis=1
    )

    df["sell_amount_norm"] = df.apply(
        lambda row: int(row["sellAmount"]) / 10 ** token_to_decimals[row["sellToken"]], axis=1
    )
    df["buy_symbol"] = df["buyToken"].map(token_to_symbol)
    df["sell_symbol"] = df["sellToken"].map(token_to_symbol)

    df["buy_amount_price"] = df.apply(lambda row: row["buy_amount_norm"] / row["sell_amount_norm"], axis=1)
    df["label"] = df["sell_symbol"] + " -> " + df["buy_symbol"]

    df["safe_value_bought"] = df.apply(lambda row: row["buy_token_price"] * row["buy_amount_norm"], axis=1)
    df["safe_value_sold"] = df.apply(lambda row: row["sell_token_price"] * row["sell_amount_norm"], axis=1)
    df["safe_value_slippage_bps"] = df.apply(
        lambda row: 1_000 * (row["safe_value_sold"] - row["safe_value_bought"]) / row["safe_value_sold"], axis=1
    )
    df["long_label"] = df["label"] + " " + df["sell_amount_norm"].astype(int).astype(str)

    df["30_min_chunk"] = pd.to_datetime(df["datetime_received"]).dt.floor("30min")
    df["60_min_chunk"] = pd.to_datetime(df["datetime_received"]).dt.floor("60min")
    df["2_hour_chunk"] = pd.to_datetime(df["datetime_received"]).dt.floor("120min")

    return df


df = load_swap_matrix_with_prices_data()

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
auto_dola_df = df[df["autopool_name"] == "autoDOLA"].reset_index(drop=True)
hourly_slippage_values = auto_dola_df.pivot(
    index="60_min_chunk", columns="long_label", values="safe_value_slippage_bps"
)
hourly_slippage_values

long_label,DOLA -> USR 100000,DOLA -> USR 150000,DOLA -> USR 200000,DOLA -> USR 250000,DOLA -> USR 50000,DOLA -> alUSD 100000,DOLA -> alUSD 150000,DOLA -> alUSD 200000,DOLA -> alUSD 250000,DOLA -> alUSD 50000,...,scrvUSD -> sUSDS 100000,scrvUSD -> sUSDS 150000,scrvUSD -> sUSDS 200000,scrvUSD -> sUSDS 250000,scrvUSD -> sUSDS 50000,scrvUSD -> sUSDe 100000,scrvUSD -> sUSDe 150000,scrvUSD -> sUSDe 200000,scrvUSD -> sUSDe 250000,scrvUSD -> sUSDe 50000
60_min_chunk,,,,,,,,,,,,,,,,,,,,,
2025-09-29 17:00:00+00:00,NaN,-3.326039,NaN,NaN,-0.838299,NaN,NaN,NaN,NaN,0.498788,...,-0.271733,NaN,-0.268837,-0.262667,NaN,NaN,NaN,-2.079377,0.071649,NaN
2025-09-29 18:00:00+00:00,4.366794,1.881816,4.387367,1.902682,1.861322,3.216416,2.992978,3.657192,3.879023,0.487961,...,-0.266014,-0.262685,-0.259785,-0.256768,-0.268730,-2.182025,-2.129854,0.175915,0.180701,-2.236041
2025-09-29 20:00:00+00:00,-1.501597,-1.491567,1.022153,-1.471240,-1.511539,0.674393,0.979707,1.217647,1.405110,0.586443,...,-0.265460,-0.241028,-0.210842,-0.233579,-0.294349,0.278871,-1.811492,-1.797853,0.314410,-1.812102
2025-09-29 21:00:00+00:00,NaN,0.346717,0.356817,0.367007,NaN,NaN,0.676924,2.096497,2.688607,0.464534,...,NaN,-0.280018,-0.264588,NaN,NaN,NaN,NaN,0.339637,NaN,-1.846562
2025-09-29 22:00:00+00:00,-0.890446,NaN,NaN,NaN,-0.900382,0.541875,NaN,NaN,NaN,NaN,...,-0.253154,NaN,NaN,-0.246249,-0.290234,-1.830709,0.320604,NaN,-1.797603,NaN
2025-09-29 23:00:00+00:00,-0.029662,-0.019660,-0.009569,0.000612,-1.055318,2.914313,3.324609,5.977186,3.356376,2.767688,...,-0.251991,-0.225576,-0.218685,-0.216913,-0.668100,-1.875661,-1.847826,-1.830633,-1.818386,-1.904025
2025-09-30 00:00:00+00:00,2.234321,NaN,1.038072,3.380974,-1.597601,NaN,-0.560916,0.529610,NaN,0.161526,...,NaN,NaN,NaN,-0.142995,-0.207924,-1.583965,NaN,NaN,NaN,NaN
2025-09-30 01:00:00+00:00,NaN,3.362384,NaN,NaN,NaN,0.359280,NaN,NaN,0.737023,NaN,...,-0.198998,-0.147158,-0.152007,NaN,NaN,NaN,-1.560309,0.634302,-1.520254,0.627435
2025-09-30 02:00:00+00:00,1.401181,1.402299,1.413212,1.422697,1.393749,2.504908,2.629519,2.743832,2.873976,2.514762,...,-0.254805,-0.252389,-0.249623,-0.253036,-0.258019,0.376379,-1.639465,-1.672697,-1.561624,0.370490


In [4]:
slippage_t_minus_1 = hourly_slippage_values.shift(1)
slippage_t_minus_2 = hourly_slippage_values.shift(2)
slippage_t_minus_3 = hourly_slippage_values.shift(3)

In [5]:
difference_between_slippage_values_t_minus1 = hourly_slippage_values - slippage_t_minus_1
difference_between_slippage_values_t_minus2 = hourly_slippage_values - slippage_t_minus_2
difference_between_slippage_values_t_minus3 = hourly_slippage_values - slippage_t_minus_3

for _df in [
    difference_between_slippage_values,
    difference_between_slippage_values_t_minus2,
    difference_between_slippage_values_t_minus3,
]:
    px.histogram(_df, color=None)

In [ ]:
break

In [6]:
for 

SyntaxError: invalid syntax (1235331270.py, line 1)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'long_label=DOLA -> USR 100000<br>value=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'DOLA -> USR 100000',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'DOLA -> USR 100000',
              'offsetgroup': 'DOLA -> USR 100000',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array([        nan,         nan,  4.36679361, -1.50159698,         nan,
                          -0.89044603, -0.02966171,  2.23432117,         nan,  1.40118096,
                          -0.50966726,         nan,  4.64987246, -0.82971022,         nan,
                           1.86031117, -0.4521872 ,  1.71410117,  2.15793433,         nan,
                    

In [ ]:
px.scatter(hourly_slippage_values)

In [ ]:
px.scatter(
    auto_dola_df,
    x="datetime_received",
    y="safe_value_slippage_bps",
    color="long_label",
    title="autoDOLA Swap Matrix Slippage Over Time",
    hover_data=[
        "aggregatorName",
        "buy_symbol",
        "sell_symbol",
        "buy_amount_norm",
        "sell_amount_norm",
        "buy_token_price",
        "sell_token_price",
    ],
)

In [ ]:
px.ecdf(
    auto_dola_df,
    x="safe_value_slippage_bps",
    title="autoDOLA Swap Matrix Slippage Distribution",
    color="aggregatorName",
)

In [ ]:
df.columns

In [ ]:
# paths = [
#     '/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/database/schema/ensure_tables_are_current/using_3rd_party/autoUSD_primary_assets_quotes_slippage_3.5_bps_1_min_sleep.csv',
#     '/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/database/schema/ensure_tables_are_current/using_3rd_party/autoUSD_primary_assets_quotes_slippage_3.5_bps.csv',
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations2.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations3.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations4.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations.csv"
# ]

# dfs = [pd.read_csv(p) for p in paths]
# df = pd.concat(dfs)

df = pd.read_csv("combined_swap_quotes.csv")
# df.to_csv("combined_swap_quotes.csv", index=True)
excluded = ["frxUSD"]
df = df[(~df["sell_symbol"].isin(excluded)) & (~df["buy_symbol"].isin(excluded))].copy()
df["datetime_received"] = pd.to_datetime(df["datetime_received"])
df = df.sort_values("datetime_received")
px.scatter(df, x="datetime_received", y="label", title="Samples over time").show()


df["long_label"] = df["label"] + " " + df["sell_amount_norm"].astype(int).astype(str)
df["15_min_chunk"] = df["datetime_received"].dt.floor("15min")

# I think we want 1 for each sample
df[["15_min_chunk", "long_label"]].value_counts().reset_index()["count"].value_counts()
# we only want the samples where there is only one price (eg skip the eariler ones)
df_unique = (
    df.groupby(["15_min_chunk", "long_label"])[
        ["datetime_received", "min_buy_amount_price", "buy_amount_price", "buy_symbol"]
    ]
    .first()
    .reset_index()
)

df_unique[["datetime_received", "long_label", "15_min_chunk", "min_buy_amount_price", "buy_amount_price"]]
buy_amount_price_over_time = df_unique.pivot(index="15_min_chunk", columns="long_label", values="buy_amount_price")

In [ ]:
def compute_bps_diff_thresholds(buy_amount_price_over_time, column: str, percentile) -> pd.DataFrame:
    one_token_prices = buy_amount_price_over_time[[column]].copy()
    one_token_prices["t-15"] = one_token_prices[column].shift(1)
    one_token_prices["t-30"] = one_token_prices[column].shift(2)
    one_token_prices["t-45"] = one_token_prices[column].shift(3)
    one_token_prices["t-60"] = one_token_prices[column].shift(4)
    one_token_prices["t-120"] = one_token_prices[column].shift(8)
    one_token_prices["t-180"] = one_token_prices[column].shift(12)
    one_token_prices["bps_diff_15_min"] = (
        (one_token_prices[column] - one_token_prices["t-15"]) / one_token_prices["t-15"] * 1_000
    )
    one_token_prices["bps_diff_30_min"] = (
        (one_token_prices[column] - one_token_prices["t-30"]) / one_token_prices["t-30"] * 1_000
    )
    one_token_prices["bps_diff_45_min"] = (
        (one_token_prices[column] - one_token_prices["t-45"]) / one_token_prices["t-45"] * 1_000
    )
    one_token_prices["bps_diff_60_min"] = (
        (one_token_prices[column] - one_token_prices["t-60"]) / one_token_prices["t-60"] * 1_000
    )
    one_token_prices["bps_diff_120_min"] = (
        (one_token_prices[column] - one_token_prices["t-120"]) / one_token_prices["t-120"] * 1_000
    )
    one_token_prices["bps_diff_180_min"] = (
        (one_token_prices[column] - one_token_prices["t-180"]) / one_token_prices["t-180"] * 1_000
    )
    thresholds = one_token_prices.filter(like="bps_diff_").abs().quantile(percentile).rename(column)
    return thresholds


percentile = 0.95


def plot_bps_thresholds_by_size():
    for size in df["sell_amount_norm"].unique().astype(int).astype(str):
        all_thresholds = []
        for i, col in enumerate(buy_amount_price_over_time.columns):
            if " " + size not in col:
                continue
            thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col, percentile)
            all_thresholds.append(thresholds)
        threshold_df = pd.concat(all_thresholds, axis=1).T
        px.histogram(threshold_df, title=f"{percentile}th percentile of bps diff for size " + size).show()


def plot_bps_diffs_by_label():
    for label in df_unique["buy_symbol"].unique():
        all_thresholds = []

        for i, col in enumerate(buy_amount_price_over_time.columns):
            if str(label) in col:
                thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col, percentile)
                all_thresholds.append(thresholds)

        if len(all_thresholds) == 0:
            print("No data for label", label)
            continue

        threshold_df = pd.concat(all_thresholds, axis=1).T
        px.ecdf(threshold_df, title=f"{percentile}th percentile of bps diff for label " + label, range_x=[0, 1]).show()


def plot_all_bps_diffs():
    all_thresholds = []
    for i, col in enumerate(buy_amount_price_over_time.columns):
        thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col, percentile)
        all_thresholds.append(thresholds)
    threshold_df = pd.concat(all_thresholds, axis=1).T
    px.ecdf(threshold_df, title=f"{percentile}th percentile of bps diff for all").show()


plot_all_bps_diffs()
plot_bps_diffs_by_label()

In [ ]:
# to

# get spot price in usd for each token at that time
# use the same block when you can, pick a block after it

# actaully you

# datetime -> block,
# get spot price of each token in usd at that block

In [ ]:
# plot_bps_thresholds_by_size()

In [ ]:
# I'm doing this wrong, I need to get the swap cost in bps, not the price change in bps
#

In [ ]:
df_unique["buy_symbol"].unique()

In [ ]:
0.0001 * 100_000

In [ ]:
[1 - (1 * 0.00002), 1 + (1 * 0.00002)]

In [ ]:
# to get,

# everything,

# what are the outliers

# On ~ September 23 - 24 ish

- 90% of prices at Time T=0 (execpt for 2 outliers) are with .2 bps of the price at T-15 for the permutations of ['USDT', 'FRAX', 'GHO', 'USDe', 'USDC', 'crvUSD']
- slippage bps setting in tokemak swap API when set to a high value gives a very small amount of bps of slippage. eg if we set slippage is 50% then min buy amount is ~50% of the price of the buy amount price



In [ ]:
100_000 - (100_000 * 0.00003), 100_000 + (100_000 * 0.00003)